# Experiment 005: Tessellation Approach

Implement the tessellation approach from egortrushin kernel:
1. Create grid patterns with nx*ny trees
2. Optimize spacing (dx, dy) and angle using SA
3. Compare to current best for each N

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json
import random
import math
import copy
from tqdm import tqdm

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0')), float(tip_y)),
            (float(top_w / Decimal('2')), float(tier_1_y)),
            (float(top_w / Decimal('4')), float(tier_1_y)),
            (float(mid_w / Decimal('2')), float(tier_2_y)),
            (float(mid_w / Decimal('4')), float(tier_2_y)),
            (float(base_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(base_y)),
            (float(-(base_w / Decimal('2'))), float(base_y)),
            (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
            (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
            (float(-(top_w / Decimal('4'))), float(tier_1_y)),
            (float(-(top_w / Decimal('2'))), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))
    
    def set_params(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def get_params(self):
        return self.center_x, self.center_y, self.angle

def has_collision(trees):
    """Check if any trees overlap"""
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                return True
    return False

def get_bounding_box_side(trees):
    """Get the side length of the bounding box"""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score(trees):
    """Calculate score for a configuration"""
    n = len(trees)
    side = get_bounding_box_side(trees)
    return side ** 2 / n

print('Functions defined.')

In [ ]:
def create_tessellation(seed_trees, nx, ny, dx, dy):
    """
    Create a tessellation by translating seed trees in a grid pattern.
    
    Args:
        seed_trees: List of seed ChristmasTree objects
        nx: Number of columns
        ny: Number of rows
        dx: Horizontal spacing
        dy: Vertical spacing
    
    Returns:
        List of ChristmasTree objects forming the tessellation
    """
    trees = []
    for seed in seed_trees:
        sx, sy, sa = seed.get_params()
        for i in range(nx):
            for j in range(ny):
                new_x = float(sx) + i * dx
                new_y = float(sy) + j * dy
                trees.append(ChristmasTree(str(new_x), str(new_y), str(sa)))
    return trees

def optimize_tessellation_spacing(seed_trees, nx, ny, target_n, max_iter=1000):
    """
    Optimize the spacing for a tessellation to minimize bounding box.
    Uses simulated annealing on dx, dy.
    """
    # Start with a reasonable spacing
    dx = 1.0
    dy = 1.0
    
    # Find initial valid spacing (no collisions)
    while True:
        trees = create_tessellation(seed_trees, nx, ny, dx, dy)
        if len(trees) >= target_n:
            trees = trees[:target_n]
        if not has_collision(trees):
            break
        dx += 0.1
        dy += 0.1
        if dx > 5.0:  # Give up if spacing is too large
            return None, float('inf')
    
    best_dx, best_dy = dx, dy
    best_score = calculate_score(trees)
    
    # SA to optimize spacing
    T = 1.0
    T_min = 0.001
    alpha = 0.99
    
    current_dx, current_dy = dx, dy
    current_score = best_score
    
    for _ in range(max_iter):
        # Perturb spacing
        new_dx = current_dx + random.uniform(-0.05, 0.05)
        new_dy = current_dy + random.uniform(-0.05, 0.05)
        
        if new_dx <= 0 or new_dy <= 0:
            continue
        
        trees = create_tessellation(seed_trees, nx, ny, new_dx, new_dy)
        if len(trees) >= target_n:
            trees = trees[:target_n]
        
        if has_collision(trees):
            continue
        
        new_score = calculate_score(trees)
        delta = new_score - current_score
        
        if delta < 0 or random.random() < math.exp(-delta / T):
            current_dx, current_dy = new_dx, new_dy
            current_score = new_score
            
            if new_score < best_score:
                best_dx, best_dy = new_dx, new_dy
                best_score = new_score
        
        T = max(T * alpha, T_min)
    
    # Return best configuration
    trees = create_tessellation(seed_trees, nx, ny, best_dx, best_dy)
    if len(trees) >= target_n:
        trees = trees[:target_n]
    
    return trees, best_score

print('Tessellation functions defined.')

In [ ]:
# Load current best solution for comparison
def load_solution(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    solution = {}
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        trees = [(row['x'], row['y'], row['deg']) for _, row in group_data.iterrows()]
        solution[n] = trees
    
    return solution

def score_config(trees_data):
    tree_list = [ChristmasTree(x, y, deg) for x, y, deg in trees_data]
    return calculate_score(tree_list)

# Load baseline
baseline = load_solution('/home/code/experiments/004_cpp_sa_optimizer/input.csv')
baseline_scores = {n: score_config(baseline[n]) for n in range(1, 201)}
baseline_total = sum(baseline_scores.values())
print(f'Baseline total score: {baseline_total:.6f}')

In [ ]:
# Try tessellation for rectangular N values
rectangular_n = [
    (4, 2, 2), (6, 2, 3), (6, 3, 2), (8, 2, 4), (8, 4, 2),
    (9, 3, 3), (10, 2, 5), (10, 5, 2), (12, 3, 4), (12, 4, 3),
    (16, 4, 4), (20, 4, 5), (20, 5, 4), (25, 5, 5),
    (36, 6, 6), (49, 7, 7), (64, 8, 8), (81, 9, 9), (100, 10, 10),
    (121, 11, 11), (144, 12, 12), (169, 13, 13), (196, 14, 14)
]

# Different seed configurations to try
seed_configs = [
    # Single tree at different angles
    [(0, 0, 0)],
    [(0, 0, 45)],
    [(0, 0, 90)],
    [(0, 0, 180)],
    # Two trees with different angles (alternating pattern)
    [(0, 0, 0), (0.5, 0, 180)],
    [(0, 0, 45), (0.5, 0, 225)],
]

improvements = []
best_solution = {n: list(baseline[n]) for n in baseline}

print('Testing tessellation for rectangular N values...')
for target_n, nx, ny in tqdm(rectangular_n):
    current_score = baseline_scores[target_n]
    best_tess_score = current_score
    best_tess_config = None
    
    for seed_config in seed_configs:
        # Create seed trees
        seed_trees = [ChristmasTree(str(x), str(y), str(a)) for x, y, a in seed_config]
        
        # Optimize tessellation
        trees, score = optimize_tessellation_spacing(seed_trees, nx, ny, target_n, max_iter=500)
        
        if trees is not None and score < best_tess_score - 1e-9:
            best_tess_score = score
            best_tess_config = [(str(t.center_x), str(t.center_y), str(t.angle)) for t in trees]
    
    if best_tess_config is not None:
        improvements.append((target_n, current_score, best_tess_score, current_score - best_tess_score))
        best_solution[target_n] = best_tess_config
        print(f'  N={target_n:3d}: {current_score:.6f} -> {best_tess_score:.6f} (improvement: {current_score - best_tess_score:.6f})')

print(f'\nFound {len(improvements)} improvements')
total_improvement = sum(delta for _, _, _, delta in improvements)
print(f'Total improvement: {total_improvement:.6f}')

In [ ]:
# Calculate new total score
new_total = sum(score_config(best_solution[n]) for n in range(1, 201))

print(f'Baseline score: {baseline_total:.6f}')
print(f'After tessellation: {new_total:.6f}')
print(f'Improvement: {baseline_total - new_total:.6f}')
print(f'Target: 68.919154')
print(f'Gap to target: {new_total - 68.919154:.6f}')

In [ ]:
# Save submission
rows = []
for n in range(1, 201):
    for i, (x, y, deg) in enumerate(best_solution[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x}',
            'y': f's{y}',
            'deg': f's{deg}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('submission.csv', index=False)
print(f'Saved submission with {len(submission_df)} rows')

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'num_improvements': len(improvements),
    'improvements_detail': [(n, old, new, delta) for n, old, new, delta in improvements]
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics')